In [1]:
import os
import sys
import talib as ta
syspath = os.path.split(os.path.abspath("."))[0]
sys.path.append(syspath)
import univ3api.simulation as sim
import univ3api.utils as utils
import importlib
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
# run if simulation file is updated
importlib.reload(sim)

HOST = "mongodb://172.16.11.81:27017/ETHDATA"
source = sim.DataUtilMongoDB(HOST)

from datetime import datetime
import time
d1 = datetime(2021,5,10, 9)
d2 = datetime(2021,11,10,9)
t1 = int(time.mktime(d1.timetuple()))
t2 = int(time.mktime(d2.timetuple()))

# print(d1.timetuple())
# print(time.mktime(d1.timetuple()))
# print(int(time.mktime(d1.timetuple())))

# swapdata = source.swap_by_time("SWAP_WETH_USDT_3000", t1, t2)
swapdata = source.swap_by_time("SWAP_USDC_WETH_3000", t1, t2)
swapdata.tail()
pricedata = swapdata[["timestamp", "sqrtPriceX96"]]
# pricedata["price"] = pricedata["sqrtPriceX96"].apply(utils.PriceConverter(18, 6).x96_to_price)
pricedata["price"] = pricedata["sqrtPriceX96"].apply(utils.PriceConverter(6, 18).x96_to_price)

In [2]:
pricedata.tail()

,timestamp,sqrtPriceX96,price
153322,1636505287,1151304179731807924804200863381341,0.000211
153323,1636505306,1153213095406308275655367738380110,0.000212
153324,1636505306,1153226560737155904671591020768514,0.000212
153325,1636505306,1153227466349991390439811165863993,0.000212
153326,1636505333,1153327366354417369770223314484875,0.000212


In [3]:
swapdata.tail()

,transactionHash,logIndex,transactionIndex,blockNumber,recipient,amount0,amount1,sqrtPriceX96,liquidity,tick,timestamp
153322,0xd05240116674df7b9a04e956c8f243617ced39dc4464...,231,145,13585408,0x8820768CAEDC113BA97dD749d948Ad0FD91c52b6,-3083890476,653166335479104327,1151304179731807924804200863381341,19661487171453067574,191691,1636505287
153323,0x651a43d186e508a7a6af0d89c8c0b53b7295cf3916e3...,2,0,13585409,0xDD56a5C540C68FC348dFa04aE6BC51BD301363a4,-2055269085762,436010152671868878848,1153213095406308275655367738380110,17444646306133960799,191724,1636505306
153324,0xe879b70dbabc82e6cae01746ce025aeb685d8454281b...,364,288,13585409,0x74de5d4FCbf63E00296fd95d33236B9794016631,-13993765084,2973750000000000000,1153226560737155904671591020768514,17444646306133960799,191724,1636505306
153325,0x7cb0b7e359f6e50025416da8cf4feb81edc0c041b9f2...,375,294,13585409,0xE592427A0AEce92De3Edee1F18E0157C05861564,-941141030,200000000000000000,1153227466349991390439811165863993,17444646306133960799,191724,1636505306
153326,0x21549a7650b1c0991c4655ee5be6067f30e06035d00f...,5,1,13585410,0xb2005EB70c799851b6f1eF4c3275E096e875CeA9,-103810118095,22062409124842680559,1153327366354417369770223314484875,17444646306133960799,191726,1636505333


In [4]:
priceDf = pricedata
priceDf['datetime'] = list(map(lambda d: datetime.fromtimestamp(d), priceDf['timestamp'] ))
priceDt = priceDf.set_index('datetime')
priceDay = priceDt.resample('D').last()
priceDay['MA50'] = priceDay['price'].rolling(50).mean()
priceDay['Trend'] = priceDay['price']>=priceDay['MA50']
priceDay.index = list(map(lambda d: d.date(), priceDay.index))
priceDay

,timestamp,sqrtPriceX96,price,MA50,Trend
2021-05-10,1620662352,1228616988312226015357337899456895,0.000240,NaN,False
2021-05-11,1620748791,1252973529377508271162254629465828,0.000250,NaN,False
2021-05-12,1620834933,1222258371118313304597344053586746,0.000238,NaN,False
2021-05-13,1620921557,1295851047853606539736005287101923,0.000268,NaN,False
2021-05-14,1621007973,1232683723756514542155780026658030,0.000242,NaN,False
...,...,...,...,...,...
2021-11-06,1636214052,1194848000641828635283333902228354,0.000227,0.000277,False
2021-11-07,1636300067,1168621881441058942578934632173765,0.000218,0.000275,False
2021-11-08,1636386947,1148661893044849945002863450619143,0.000210,0.000273,False
2021-11-09,1636473513,1144150487469488176795506712967075,0.000209,0.000271,False


In [5]:
trendDict = dict(priceDay['Trend'])
priceDt['date'] = list(map(lambda d: d.date(), priceDt.index))
trendList = []
for i, v in priceDt.iterrows():
    trendList.append(trendDict[v['date']])
print(f'len of trendList:{len(trendList)}')
priceDt['trend'] = trendList

len of trendList:153327


In [6]:
priceDt

,timestamp,sqrtPriceX96,price,date,trend
datetime,,,,,
2021-05-10 09:17:52,1620609472,1267554519342589161168659854845874,0.000256,2021-05-10,False
2021-05-10 09:23:50,1620609830,1267498129839936824485169221023868,0.000256,2021-05-10,False
2021-05-10 09:24:49,1620609889,1267496927736336268048970777338285,0.000256,2021-05-10,False
2021-05-10 09:24:49,1620609889,1267496466129159558106023913391909,0.000256,2021-05-10,False
2021-05-10 09:24:54,1620609894,1267269382521328460205280017540103,0.000256,2021-05-10,False
...,...,...,...,...,...
2021-11-10 08:48:07,1636505287,1151304179731807924804200863381341,0.000211,2021-11-10,False
2021-11-10 08:48:26,1636505306,1153213095406308275655367738380110,0.000212,2021-11-10,False
2021-11-10 08:48:26,1636505306,1153226560737155904671591020768514,0.000212,2021-11-10,False


In [7]:
pricedata_res = pd.read_csv('../../data/usdceth/pricedata_res.csv',index_col=0) 
#pricedata_res 其实是处理过后的priceDt
pricedata_res

,timestamp,sqrtPriceX96,price,SmaLowerLma,VolLowerQuantile75,CloseLowerMA,VolHigherQuantile50TwoSigma,revoke_pos
datetime,,,,,,,,
2021-05-10 09:17:52,1620609472,1267554519342589161168659854845874,0.000216,0.0,0.0,0.0,0.0,1.0
2021-05-10 09:23:50,1620609830,1267498129839936824485169221023868,0.000049,0.0,0.0,0.0,0.0,1.0
2021-05-10 09:24:49,1620609889,1267496927736336268048970777338285,0.000050,0.0,0.0,0.0,0.0,1.0
2021-05-10 09:24:49,1620609889,1267496466129159558106023913391909,0.000051,0.0,0.0,0.0,0.0,1.0
2021-05-10 09:24:54,1620609894,1267269382521328460205280017540103,0.000154,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...
2021-09-01 08:48:00,1630457280,1355527382349696291973070296916222,0.000057,1.0,0.0,0.0,0.0,0.0
2021-09-01 08:48:36,1630457316,1355633495651790988012240522812889,0.000193,1.0,0.0,0.0,0.0,0.0
2021-09-01 08:49:15,1630457355,1356496723249880871370773228843790,0.000092,1.0,0.0,0.0,0.0,0.0


In [8]:
pricedata_res['price'] = priceDt['price']
pricedata_res

ValueError: cannot reindex from a duplicate axis

In [ ]:
init_price = pricedata_res["price"][0]

decimal0 = 6 #USDC的精度
decimal1 = 18 #ETH的精度

init_amount0 = 1000000*1e6 #初始持有USDC的数量
init_amount1 = 0 #初始持有ETH的数量
print('init_price =',init_price)

In [ ]:
import Strategy_usdceth_vol
importlib.reload(utils)
importlib.reload(sim)
importlib.reload(Strategy_usdceth_vol)
strategy = Strategy_usdceth_vol.HoldStrategy(init_amount0, init_amount1, decimal0, decimal1, sim.PoolFee.low, True)
strategy.init(swapdata.iloc[0].to_dict())

# strategy.run(swapdata, priceDt)
strategy.run(swapdata, pricedata_res)

report = strategy.report()

In [ ]:
report.positions[1].get_balance(plain=True, draw_plot=True)

In [ ]:
report.positions[2].get_balance(plain=True, draw_plot=True)

In [ ]:
report.positions[3].get_balance(plain=True, draw_plot=True)

In [ ]:
total_balance = report.total_balance(draw_plot=True, plain=True)

In [ ]:
price = pricedata.drop_duplicates("timestamp").set_index("timestamp")["price"]
price
usdc_balance = total_balance.set_index("timestamp")[["amount0", "cumFee0"]].sum(axis=1)
eth_balance = total_balance.set_index("timestamp")[["amount1", "cumFee1"]].sum(axis=1) / price
eth_balance+usdc_balance
# eth_balance.plot()
# usdt_balance.plot()
tb_Timing = (eth_balance+usdc_balance)
tb_Timing_origin = tb_Timing.copy(deep=True)
time_to_print = list(map(time.localtime,tb_Timing.index.tolist()))
time_to_print_2  = [time.strftime('%Y-%m-%d %H:%M:%S', i) for i in time_to_print]
# time_to_print_2  = [time.strftime('%Y-%m-%d', i) for i in time_to_print]
tb_Timing.index = time_to_print_2
tb_Timing.plot(rot = 45,grid=True)
tb_Timing.to_csv('../../data/usdceth/tb_timing.csv')
tb_Timing_origin.to_csv('../../data/usdceth/tb_timing_origin.csv')

In [ ]:
assets_value = (total_balance["amount1"] / 10**decimal1)/pricedata.set_index("timestamp")["price"] + total_balance["amount0"] / 10**decimal0
assets_value.plot()
# pricedata.set_index("timestamp")[["price"]].plot()

In [ ]:
assets_value